# Pytorch
## 加载数据
dataset -> dataloader

In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import os

class my_data(Dataset):

    def __init__(self, root_dir, label_dir) -> None:
        super().__init__()
        self.root_dir = root_dir
        self.label_dir = label_dir
        self.path = os.path.join(self.root_dir, self.label_dir)
        self.img_path = os.listdir(self.path)
    
    def __getitem__(self, index):
        img_name = self.img_path[index]
        img_item_path = os.path.join(self.root_dir, self.label_dir, img_name)
        img = Image.open(img_item_path)
        label = self.label_dir
        return img, label
    
    def __len__(self):
        return len(self.img_path)

root_dir = '/Users/chen/Desktop/练手数据集/train'
ants_image_dir = 'ants_image'
ants_dataset = my_data(root_dir, ants_image_dir)

img, label = ants_dataset[0]
img

# len(ants_dataset)

## Tensorboard and Transform

### 使用tensorboard
### 使用transform转换成tensor数据格式

In [1]:
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from PIL import Image


writer = SummaryWriter('/Users/chen/Desktop/练手数据集/train/logs')
image_path = ('/Users/chen/Desktop/练手数据集/train/ants_image/0013035.jpg')
image_PIL = Image.open(image_path)
img_array = np.array(image_PIL)

writer.add_image('test', img_array, 2,dataformats="HWC")
# for i in range(100):

#     writer.add_scalar('y=x',3*i,i)  #tag, scalar_value(y), global_step(x)


writer.close()

In [ ]:
# 使用tensorboard

from PIL import Image
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms

img_path = '/Users/chen/Desktop/练手数据集/train/ants_image/5650366_e22b7e1065.jpg'
img = Image.open(img_path)

writer = SummaryWriter('/Users/chen/Desktop/练手数据集/train/logs')

# toTensor
tensor_trans = transforms.ToTensor()
ten_img = tensor_trans(img)

writer.add_image('tor',ten_img)

writer.close()

In [ ]:
from torchvision import transforms
from PIL import Image

img = Image.open('/Users/chen/Desktop/练手数据集/train/ants_image/6240329_72c01e663e.jpg')

# normalize
print(ten_img[0][0][0])
tran = transforms.Normalize([0.5,1,0.1],[0.2,0.3,1])
img_nor = tran(ten_img)
print(img_nor[0][0][0])

writer.add_image('Nor',img_nor)

writer.close()



In [ ]:
# resize - PIL
print(img.size)
trans_size = transforms.Resize((512,512))
img_resized = trans_size(img)
print(img_resized.size)

# PIL - resize - toTensor
img_resized = trans_size(img)
print(img_resized.size)
type(img_resized)

img_resized = tensor_trans(img_resized)
type(img_resized)

# compose 连续调用方法集合
trans_resized2 = transforms.Resize(800)
trans_compose = transforms.Compose([trans_resized2,tensor_trans])
img_resized2 = trans_compose(img)

writer.add_image('resized2',img_resized2)

writer.close()


### 使用torchvision中的数据集

In [2]:
import torchvision

train_set = torchvision.datasets.CIFAR10(root='./dataset',train=True,download=True)
test_set = torchvision.datasets.CIFAR10(root='./dataset',train=False,download=True)

train_set[0]


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./dataset/cifar-10-python.tar.gz to ./dataset
Files already downloaded and verified


(<PIL.Image.Image image mode=RGB size=32x32 at 0x7FF721239B50>, 6)

In [ ]:
from turtle import forward
from torch import nn
import torch

class Tudui(nn.Module):
    def __init__(self) -> None:
        super().__init__()

    def forward(self,input):
        output = input + 1
        return output

tudui = Tudui()
x = torch.tensor(1.0)
opt = tudui(x)
opt

## Convolutional layers

In [10]:
import torch
import torch.nn.functional as F

input = torch.tensor(
    [
        [1,2,0,3,1],
        [0,1,2,3,1],
        [1,2,1,0,0],
        [5,2,3,1,1],
        [2,1,0,1,1]
    ]
)

kernel = torch.tensor(
    [
        [1,2,1],
        [0,1,0],
        [2,1,0]
    ]
)

input = torch.reshape(input,(1,1,5,5))
kernel = torch.reshape(kernel,(1,1,3,3))

print(input.shape)
print(kernel.shape)

output = F.conv2d(input,kernel,stride = 1)
print(output)

output2 = F.conv2d(input,kernel,stride = 2)
print(output2)

output3 = F.conv2d(input,kernel,stride = 1,padding=1)
print(output3)



torch.Size([1, 1, 5, 5])
torch.Size([1, 1, 3, 3])
tensor([[[[10, 12, 12],
          [18, 16, 16],
          [13,  9,  3]]]])
tensor([[[[10, 12],
          [13,  3]]]])
tensor([[[[ 1,  3,  4, 10,  8],
          [ 5, 10, 12, 12,  6],
          [ 7, 18, 16, 16,  8],
          [11, 13,  9,  3,  4],
          [14, 13,  9,  7,  4]]]])


In [8]:
import torchvision
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.nn import Conv2d
from torch.utils.tensorboard import SummaryWriter


dataset = torchvision.datasets.CIFAR10('./dataset',train=False,transform=torchvision.transforms.ToTensor(),download=True)
dataloader = DataLoader(dataset,batch_size=64)

class Tudui(nn.Module):
    def __init__(self) -> None:
        super(Tudui,self).__init__()
        self.conv1 = Conv2d(in_channels=3,out_channels=6,kernel_size=3,stride=1,padding=0)
    
    def forward(self,x):
        x = self.conv1(x)
        return x

tudui = Tudui()
step = 0
writer = SummaryWriter('./logs')
for data in dataloader:
    imgs, targets = data
    output = tudui(imgs)
    writer.add_images('input',imgs,step)
    output = torch.reshape(output,(-1,3,30,30))
    writer.add_images('output',output,step)

    step += 1


Files already downloaded and verified


## Pooling layer
maxpool

In [13]:
import torch
import torchvision
from torch import nn
from torch.nn import MaxPool2d
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

dataset = torchvision.datasets.CIFAR10("./dataset", train=False, download=True,
                                       transform=torchvision.transforms.ToTensor())

dataloader = DataLoader(dataset, batch_size=64)

class Tudui(nn.Module):
    def __init__(self):
        super(Tudui, self).__init__()
        self.maxpool1 = MaxPool2d(kernel_size=3, ceil_mode=False)

    def forward(self, input):
        output = self.maxpool1(input)
        return output

tudui = Tudui()

writer = SummaryWriter("./logs_maxpool")
step = 0

for data in dataloader:
    imgs, targets = data
    writer.add_images("input", imgs, step)
    output = tudui(imgs)
    writer.add_images("output", output, step)
    step = step + 1

writer.close()



Files already downloaded and verified


## No linear activation / Sigmoid